### Counting number of the mutual friends

For each user having ID in the column userId count the amount of his / her common friends with each other user having ID in the column userId.

Print 49 pairs of the users having the largest amount of common friends, ordered in descending order first by the common friends count , then by id of user1 and finally by id of user 2. The format is following: "count user1 user2"7

Example:

234 54719 767867

120 54719 767866

97 50787 327676

To solve this task use the algorithm described in the last video of lesson 1. The overall plan could look like this:

1. Create a new column “friend” by exploding of column “friends” (like in the demo iPython notebook)
2. group the resulting dataframe by the column “friend” (like in the demo iPython notebook)
3. create a column “users” by collecting all users with the same id in the column “friend” together (like in the demo iPython notebook)
4. sort the elements in the column “users” by the function sort_array
5. filter only the rows which have more than 1 element in the column “users”
6. for each row emit all possible ordered pairs of users from the column “users” (tip: write a user defined function for this)
7. count the number of times each pair has appeared
8. with the help of the window function (like in the demo python notebook) select 49 pairs of users who have the biggest amount of common friends
The sample dataset is located at /data/graphDFSample.

The part of the result on the sample dataset:

...
3044 21864412 51640390
3021 17139850 51640390
3010 14985079 51640390
2970 17139850 21864412
2913 20158643 27967558
...

Notes

The system grades standard output and error streams from the last non-empty cell. If you have clicked "Open tool" and received "404" or "405" error, please reload the page from the Coursera interface.

If you want to deploy the environment on your own machine, please use bigdatateam/spark-course2 Docker container.

In [1]:
from pyspark.sql.functions import explode, collect_list, size, col, row_number, sort_array,udf, count, desc
from pyspark.sql import Window, types
from itertools import combinations
from pyspark.sql import SparkSession

In [2]:
graphPath = "/data/graphDFSample"
spark_session = SparkSession.builder.enableHiveSupport().master("yarn").getOrCreate()

In [3]:
# 1. Create a new column “friend” by exploding of column “friends” (like in the demo iPython notebook)
# 2. group the resulting dataframe by the column “friend” (like in the demo iPython notebook)
# 3. create a column “users” by collecting all users with the same id in the column “friend” together (like in the demo iPython notebook)
# 4. sort the elements in the column “users” by the function sort_array
# 5. filter only the rows which have more than 1 element in the column “users”

reversedGraph = (
    spark_session.read.parquet(graphPath)
    .withColumn("friend", explode('friends'))
    .groupBy("friend")
    .agg(collect_list("user").alias("users"))
    .withColumn("users", sort_array('users'))
    .where(size('users') > '1')
                )

In [4]:
reversedGraph.printSchema()

root
 |-- friend: integer (nullable = true)
 |-- users: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [5]:
# 6. for each row emit all possible ordered pairs of users from the column “users” (tip: write a user defined function for this)

def combinatorix_pairs(users):
    return combinations(sorted(users), 2)

pair_schema = types.StructType([
    types.StructField("frnd1", types.IntegerType(), False),
    types.StructField("frnd2", types.IntegerType(), False)
])

udf_combinatorix_pairs = udf(combinatorix_pairs, types.ArrayType(pair_schema))

In [6]:
# 7. count the number of times each pair has appeared
pair_counts = (reversedGraph.withColumn("pairs", udf_combinatorix_pairs('users'))
         .withColumn("pair", explode('pairs'))
         .groupBy("pair")
         .agg(count("pair").alias("pair_count"))
        )

In [7]:
# with the help of the window function (like in the demo python notebook), select 49 pairs of users who have the biggest amount of common friends
final_results = (pair_counts.select(col("pair_count"), "pair.*")
           .orderBy(desc("pair_count"), desc("frnd1"), desc("frnd2"))
           .limit(49)
          ).collect()

for pair, frnd1, frnd2 in final_results:
    print("{} {} {}".format(pair, frnd1, frnd2))

3206 27967558 42973992
3130 20158643 42973992
3066 22582764 42973992
3044 21864412 51640390
3021 17139850 51640390
3010 14985079 51640390
2970 17139850 21864412
2913 20158643 27967558
2903 22280814 51151280
2870 23848749 51640390
2855 20158643 22582764
2849 20158643 44996025
2846 22280814 42973992
2784 21864412 23848749
2779 31964081 51640390
2776 39205988 51640390
2754 17139850 23848749
2749 22582764 27967558
2728 50561859 51640390
2724 15485897 51640390
2700 28135661 42973992
2655 22280814 27967558
2653 42973992 43548989
2639 26755857 51640390
2621 14635589 51640390
2608 15485897 17139850
2606 17139850 26755857
2601 21864412 39205988
2600 8406745 51640390
2599 37735419 51640390
2597 20158643 28135661
2585 40003405 42973992
2585 21864412 31964081
2581 27967558 43548989
2579 23848749 31964081
2578 27967558 28135661
2578 15485897 21864412
2577 42973992 64755069
2574 51151280 57077210
2573 20158643 43548989
2566 21864412 26755857
2564 22280814 64755069
2561 42973992 44996025
2556 1713985

In [8]:
# window = Window.orderBy(col("users_size").desc())
    
# top50 = reversedGraph.withColumn("row_number", row_number().over(window)) \
#             .filter(col("row_number") < 50) \
#             .select(col("friend"), col("users_size")) \
#             .orderBy(col("users_size").desc()) \
#             .collect()

In [9]:
# spark_session.stop()

#### Final notice:

1. Please take into account that you must __not__ redirect __stderr__ to anywhere. Hadoop, Hive, and Spark print their logs to stderr and the Grading system also reads and analyses it.

1. During checking the code from the notebook, the system runs all notebook's cells and reads the output of only the last filled cell. It is clear that any exception should not be thrown in the running cells. If you decide to write some text in a cell, you should change the style of the cell to Markdown (Cell -> Cell type -> Markdown).

1. The Grader takes into account the output from the sample dataset you have in the notebook. Therefore, you have to "Run All" cells in the notebook before you send the ipynb solution.

1. The name of the notebook must contain only Roman letters, numbers and characters “-” or “_”. For example, Windows adds something like " (2)" (with the leading space) at the end of a filename if you try to download a file with the same name. This is a problem, because you will have a space character and curly braces "(" and ")". 